In [1]:
import sys
# append the path of the parent directory
sys.path.append("..")

import torch
import numpy as np
import random

from model import SmallBasicUNet, MediumBasicUNet, MediumUNetPlus, CNNLSTM  # Replace with your actual model class name if different
from dataset import DebrisFlowDataset, Augmentation  # Replace with your actual dataset class name if different
from dataset import compute_channel_scaling_params, set_channel_scaling_params_to_dataset
from train import Trainer

from torch.optim import Adam
from torch.utils.data import DataLoader, Subset, random_split


from viz import ArrayVisualizer

import matplotlib.pyplot as plt

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)


cuda


In [2]:
def set_seed(seed_value):
    """Set seed for reproducibility."""
    np.random.seed(seed_value)  # NumPy
    torch.manual_seed(seed_value)  # PyTorch
    random.seed(seed_value)  # Python's built-in random module
    
    # minimise non-deterministic GPU behaviour
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Usage
set_seed(42) 

In [3]:
LEARNING_RATE = 0.0001
SEQUENCE_LENGTH = 5
BATCH_SIZE = 8
LSTM_HIDDEN_SIZE = 256
LSTM_LAYERS = 8
NUM_EPOCHS = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# # Load the complete dataset
# full_dataset = DebrisFlowDataset(r'/home/tom/Downloads/data_small_temp', input_array_size=256, sequence_length=SEQUENCE_LENGTH)

# # Split the dataset into training, validation, and test sets
# train_size = int(0.7 * len(full_dataset))
# val_size = int(0.15 * len(full_dataset))
# test_size = len(full_dataset) - train_size - val_size
# train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# # Create a DataLoader for the training dataset to compute scaling parameters
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)




In [4]:
def train_val_test_split_by_model(full_dataset, train_frac=0.7, val_frac=0.15, test_frac=0.15):
    # Ensure fractions sum to 1
    assert np.isclose(train_frac + val_frac + test_frac, 1.0), "The fractions must sum to 1."

    # Extract the model IDs from the dataset
    model_ids = [data_info[0] for data_info in full_dataset.data_info]
    unique_model_ids = list(set(model_ids))

    # Shuffle the unique model IDs
    np.random.shuffle(unique_model_ids)

    # Calculate the number of models for each set
    num_models = len(unique_model_ids)
    num_train = int(np.floor(train_frac * num_models))
    num_val = int(np.floor(val_frac * num_models))

    # The rest goes to the test set to account for any rounding issues
    num_test = num_models - num_train - num_val

    # Split the model IDs into train, val, and test sets based on the calculated numbers
    train_model_ids = set(unique_model_ids[:num_train])
    val_model_ids = set(unique_model_ids[num_train:num_train + num_val])
    test_model_ids = set(unique_model_ids[num_train + num_val:])

    # Verify that we have non-empty validation and test sets
    assert len(val_model_ids) > 0, "Validation set is empty. Adjust the fractions or dataset size."
    assert len(test_model_ids) > 0, "Test set is empty. Adjust the fractions or dataset size."

    # Create indices for train, val, and test sets
    train_indices = [i for i, id in enumerate(model_ids) if id in train_model_ids]
    val_indices = [i for i, id in enumerate(model_ids) if id in val_model_ids]
    test_indices = [i for i, id in enumerate(model_ids) if id in test_model_ids]

    # Create subsets for train, val, and test
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)
    test_dataset = Subset(full_dataset, test_indices)

    return train_dataset, val_dataset, test_dataset


# Load the complete dataset
# full_dataset = DebrisFlowDataset(r'/home/tom/Downloads/data_small_temp', input_array_size=256, sequence_length=SEQUENCE_LENGTH)
full_dataset = DebrisFlowDataset(r'/home/tom/repos/dyna-landslide-surrogate/data', input_array_size=256, sequence_length=SEQUENCE_LENGTH)

# Split the dataset into training, validation, and test sets
train_dataset, val_dataset, test_dataset = train_val_test_split_by_model(full_dataset, train_frac=0.7, val_frac=0.15)

# Create a DataLoader for the training dataset to compute scaling parameters
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
# len(full_dataset)

In [6]:
# full_dataset.data_info

In [7]:
# # Access the file information in the full_dataset
# def get_data_info(dataset, indices):
#     # Retrieve the data info for the given indices
#     return [dataset.data_info[idx][0] for idx in indices]


# def collect_unique_filenames(data_info, indices):
#     elevation_files = set()
#     thickness_files = set()
#     velocity_files = set()

#     for idx in indices:
#         _, data = data_info[idx]
#         elevation_files.add(data['elevation_file'])
#         thickness_files.update(data['thickness_files'])
#         velocity_files.update(data['velocity_files'])

#     return elevation_files, thickness_files, velocity_files


# # Retrieve the data info for the training, validation, and test datasets
# train_data_info = get_data_info(full_dataset, train_dataset.indices)
# val_data_info = get_data_info(full_dataset, val_dataset.indices)
# test_data_info = get_data_info(full_dataset, test_dataset.indices)

# unique_elevation_files, unique_thickness_files, unique_velocity_files = collect_unique_filenames(full_dataset.data_info, train_dataset.indices)

# # Print out debugging information for unique elevation files
# print(f"Number of unique elevation files: {len(unique_elevation_files)}")
# print("Examples of elevation files:")
# for filename in list(unique_elevation_files)[:5]:  # Print out the first 5 examples
#     print(filename)

# # Print out debugging information for unique thickness files
# print(f"\nNumber of unique thickness files: {len(unique_thickness_files)}")
# print("Examples of thickness files:")
# for filename in list(unique_thickness_files)[:5]:  # Print out the first 5 examples
#     print(filename)

# # Print out debugging information for unique velocity files
# print(f"\nNumber of unique velocity files: {len(unique_velocity_files)}")
# print("Examples of velocity files:")
# for filename in list(unique_velocity_files)[:5]:  # Print out the first 5 examples
#     print(filename)

# # # Now you can print or inspect the data info
# # print('Train dataset data len:', len(train_data_info))
# # print('Validation dataset data len:', len(val_data_info))
# # print('Test dataset data len:', len(test_data_info))
# # print('Train dataset data info:', train_data_info)
# # print('Validation dataset data info:', val_data_info)
# # print('Test dataset data info:', test_data_info)

# # print(len(train_data_info))

In [8]:
# # Convert lists to sets for faster operation
# train_data_set = set(train_data_info)
# val_data_set = set(val_data_info)
# test_data_set = set(test_data_info)

# # Assert that there are no common elements between any two sets
# assert train_data_set.isdisjoint(val_data_set), "Training and validation sets have overlapping elements."
# assert train_data_set.isdisjoint(test_data_set), "Training and test sets have overlapping elements."
# assert val_data_set.isdisjoint(test_data_set), "Validation and test sets have overlapping elements."

# print("All datasets are disjoint. No overlapping elements found.")

In [9]:
# full_dataset.data_info[:2]

In [10]:
# # Initialize statistics
# batch_sizes = []
# shapes = {}
# min_vals = {}
# max_vals = {}
# mean_vals = {}
# var_vals = {}


# # Iterate over the DataLoader
# for i, (inputs, targets) in enumerate(train_loader):
#     # Record batch size
#     batch_sizes.append(inputs.size(0))
    
#     # Check shapes and record them
#     for j, input_tensor in enumerate(inputs):
#         if i == 0:  # Initialize stats dictionaries on the first batch
#             shapes[j] = []
#             min_vals[j] = []
#             max_vals[j] = []
#             mean_vals[j] = []
#             var_vals[j] = []
        
#         shapes[j].append(input_tensor.shape)
#         min_vals[j].append(input_tensor.min().item())
#         max_vals[j].append(input_tensor.max().item())
#         mean_vals[j].append(input_tensor.mean().item())
#         var_vals[j].append(input_tensor.var().item())
    


# # Compute global statistics
# total_batches = len(batch_sizes)
# global_batch_size = sum(batch_sizes) / len(batch_sizes)
# global_min_vals = {k: min(v) for k, v in min_vals.items()}
# global_max_vals = {k: max(v) for k, v in max_vals.items()}
# global_mean_vals = {k: sum(v) / len(v) for k, v in mean_vals.items()}
# global_var_vals = {k: sum(v) / len(v) for k, v in var_vals.items()}

# # Print statistics
# print(f"Total number of batches: {total_batches}")
# print(f"Average batch size: {global_batch_size}")
# print(f"Min values per channel: {global_min_vals}")
# print(f"Max values per channel: {global_max_vals}")
# print(f"Mean values per channel: {global_mean_vals}")
# print(f"Variance values per channel: {global_var_vals}")

# # If you want to see individual batch stats:
# print(f"Batch sizes: {batch_sizes}")
# for channel, channel_shapes in shapes.items():
#     print(f"Shapes for channel {channel}: {channel_shapes}")




In [11]:
# # Set the index of the batch you want to retrieve
# desired_batch_index = 3  # Replace with the batch index you want to retrieve

# # Variables to store the inputs and targets for the desired batch
# desired_inputs = None
# desired_targets = None

# # Iterate over the DataLoader and retrieve the desired batch
# for i, (inputs, targets) in enumerate(train_loader):
#     if i == desired_batch_index:
#         desired_inputs = inputs
#         desired_targets = targets
#         break  # Exit the loop after retrieving the desired batch

In [12]:
# desired_targets[0].shape

In [13]:
# import matplotlib.pyplot as plt

# def plot_input_channels_side_by_side(tensor, index, title_prefix):
#     """Plot channels 1 and 2 for a given sequence index from a 5D tensor side by side."""
#     fig, axs = plt.subplots(1, tensor.size(1), figsize=(15, 5))  # tensor.size(1) is the sequence length
#     for seq_num in range(tensor.size(1)):  # Loop through the sequence length
#         for channel in range(1, 3):  # Plot only channels 1 and 2
#             channel_data = tensor[index, seq_num, channel, :, :].cpu().detach().numpy()
#             axs[seq_num].imshow(channel_data, cmap='viridis')
#             axs[seq_num].set_title(f'{title_prefix} - Seq {seq_num} - Channel {channel}')
#             axs[seq_num].axis('off')
#     plt.show()

# def plot_target_channels_side_by_side(tensor, index, channel_titles):
#     """Plot the two target channels side by side."""
#     fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 2 target channels
#     for channel, title in enumerate(channel_titles):
#         channel_data = tensor[index, :, :, channel].cpu().detach().numpy()
#         axs[channel].imshow(channel_data, cmap='viridis')
#         axs[channel].set_title(f'{title} - Data Item {index}')
#         axs[channel].axis('off')
#     plt.show()

# # Set the index of the sequence you want to plot within the retrieved batch
# sequence_index_to_plot = 0  # Replace with the sequence index you want to plot within the batch

# # Plot the input channels for the specified sequence index within the retrieved batch
# if desired_inputs is not None:
#     plot_input_channels_side_by_side(desired_inputs, sequence_index_to_plot, title_prefix="Input")

# # Plot the target channels for the specified sequence index within the retrieved batch
# if desired_targets is not None:
#     plot_target_channels_side_by_side(desired_targets, sequence_index_to_plot, channel_titles=["Target Velocity", "Target Thickness"])

In [14]:
# print(f"Total items in dataset: {len(train_loader.dataset)}")
# print(f"Batch size: {train_loader.batch_size}")

In [15]:
# # Compute scaling parameters from the training DataLoader
# median_vals, mad_vals = compute_channel_scaling_params(unique_elevation_files, unique_thickness_files, unique_velocity_files)

# print(median_vals)

# print(mad_vals)

In [16]:
import torch.nn as nn
import torch.nn.functional as F

class HybridMSELoss(nn.Module):
    def __init__(self, non_zero_weight=0.8):
        super(HybridMSELoss, self).__init__()
        self.non_zero_weight = non_zero_weight

    def forward(self, input, target):
        # Overall MSE
        mse_loss = F.mse_loss(input, target, reduction='mean')
        
        # Masked MSE for non-zero elements
        mask = target != 0
        masked_input = input[mask]
        masked_target = target[mask]
        
        if masked_target.nelement() == 0:  # Handle case with no non-zero elements
            non_zero_mse_loss = 0
        else:
            non_zero_mse_loss = F.mse_loss(masked_input, masked_target, reduction='mean')
        
        # Combined loss
        combined_loss = (self.non_zero_weight * non_zero_mse_loss +
                         (1 - self.non_zero_weight) * mse_loss)
        return combined_loss
    
class AsymmetricMSELoss(nn.Module):
    def __init__(self, false_negative_weight=2.0, false_positive_weight=0.5):
        super(AsymmetricMSELoss, self).__init__()
        self.false_negative_weight = false_negative_weight
        self.false_positive_weight = false_positive_weight

    def forward(self, input, target):
        # Calculate the squared error
        squared_error = (input - target) ** 2

        # Create masks for false negatives and false positives
        false_negatives = (input == 0) & (target != 0)
        false_positives = (input != 0) & (target == 0)

        # Apply weights to the errors
        loss = squared_error
        loss[false_negatives] *= self.false_negative_weight
        loss[false_positives] *= self.false_positive_weight

        # Calculate the final loss as the mean of the weighted errors
        return loss.mean()

In [17]:
# Apply the scaling parameters to the original dataset
# train, val, test are type "subset" when using random_split (which just stores a list of idx)
# set_channel_scaling_params_to_dataset(full_dataset, median_vals, mad_vals)

# Now you can create DataLoaders for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


# Model, criterion, and optimizer
unet = MediumUNetPlus(in_channels=3, out_channels=2)
cnn_lstm_model = CNNLSTM(unet=unet, lstm_hidden_size=LSTM_HIDDEN_SIZE, lstm_layers=LSTM_LAYERS)
# criterion = torch.nn.MSELoss()  
criterion = HybridMSELoss(non_zero_weight=0.75)
#criterion = AsymmetricMSELoss(false_negative_weight=10.0, false_positive_weight=1.0)
optimizer = torch.optim.Adam(cnn_lstm_model.parameters(), lr=LEARNING_RATE)


# If you have multiple GPUs and want to use them, wrap your model with DataParallel
if torch.cuda.device_count() > 1:
    print(f"Let's use {torch.cuda.device_count()} GPUs!")
    # This will split your data automatically and send job orders to multiple GPUs
    cnn_lstm_model = torch.nn.DataParallel(cnn_lstm_model)

# Make sure to call .to(device) to move your model to the default device
cnn_lstm_model.to('cuda')  # Assuming that you are using CUDA


# Instantiate augmentation
# TODO - CHECK THIS IS WORKING
augmentation = Augmentation()

# Trainer
trainer = Trainer(cnn_lstm_model, criterion, optimizer, device)

# Training and validation
trainer.train(train_loader, val_loader, NUM_EPOCHS)

# Testing
test_loss = trainer.test(test_loader)

Let's use 2 GPUs!
Epoch 1/100 - Train Loss: 17.1027, Validation Loss: 18.5181
Epoch 2/100 - Train Loss: 15.7926, Validation Loss: 18.3781
Epoch 3/100 - Train Loss: 15.3065, Validation Loss: 18.3237
Epoch 4/100 - Train Loss: 14.9084, Validation Loss: 17.8427
Epoch 5/100 - Train Loss: 14.4664, Validation Loss: 17.7634
Epoch 6/100 - Train Loss: 14.1779, Validation Loss: 18.0550
Epoch 7/100 - Train Loss: 13.9117, Validation Loss: 17.7162
Epoch 8/100 - Train Loss: 13.7853, Validation Loss: 17.8330
Epoch 9/100 - Train Loss: 13.5616, Validation Loss: 17.8448
Epoch 10/100 - Train Loss: 13.4841, Validation Loss: 18.4818
Epoch 11/100 - Train Loss: 13.3026, Validation Loss: 17.9241
Epoch 12/100 - Train Loss: 13.1586, Validation Loss: 18.0550
Epoch 13/100 - Train Loss: 13.1453, Validation Loss: 17.8848
Epoch 14/100 - Train Loss: 12.9956, Validation Loss: 18.1634
Epoch 15/100 - Train Loss: 12.7929, Validation Loss: 17.9054
Epoch 16/100 - Train Loss: 12.7272, Validation Loss: 18.0273
Epoch 17/100 - 

KeyboardInterrupt: 

In [18]:
import numpy as np
import matplotlib.pyplot as plt

# For MINMAX SCALED
# def inverse_transform(data, min_val, max_val):
#     """
#     Inverse transform the data from the scaled range back to the original range.
    
#     Args:
#         data (np.ndarray): The data to inverse transform.
#         min_val (float): The minimum value of the original range.
#         max_val (float): The maximum value of the original range.
    
#     Returns:
#         np.ndarray: The inverse transformed data.
#     """
#     return data * (max_val - min_val) + min_val

def inverse_transform(data, median_val, iqr_val):
    """
    Inverse transform the data from the scaled range back to the original range using median and IQR.
    
    Args:
        data (np.ndarray): The data to inverse transform.
        median_val (float): The median value used for scaling.
        iqr_val (float): The IQR value used for scaling.
    
    Returns:
        np.ndarray: The inverse transformed data.
    """
    return (data * iqr_val) + median_val

def plot_array(array, ax, title, label, cmap='viridis', vmin=None, vmax=None):
    """
    Plot a 2D array as an image with an additional label.
    
    Args:
        array (np.ndarray): The 2D array to plot.
        ax (matplotlib.axes.Axes): The matplotlib axes to plot on.
        title (str): The title of the plot.
        label (str): The additional label to display on the plot.
        cmap (str): The colormap to use (default: 'viridis').
        vmin (float): The minimum value for the colorbar (default: None).
        vmax (float): The maximum value for the colorbar (default: None).
    """
    im = ax.imshow(array, cmap=cmap, vmin=vmin, vmax=vmax)
    ax.set_title(f"{title}\n{label}")
    ax.axis('off')
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)


indices = [10,20,30,40,50,60]

for i in indices:
    # Testing
    print(i)
    test_loss, predicted, target = trainer.test(test_loader, return_indices=i)
    print(f'Test Loss: {test_loss:.4f}')

    model_name = type(trainer.model).__name__
    model_state_id = id(trainer.model.state_dict())

    plot_label = f"Model: {model_name}, State ID: {model_state_id}"

    #  predicted and target values without inverse
    predicted_thickness_inverse = predicted[..., 0]
    target_thickness_inverse = target[..., 0]
    predicted_velocity_inverse = predicted[..., 1]
    target_velocity_inverse = target[..., 1]
    
    # Plot the predicted thickness, target thickness, and their difference
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    plot_array(predicted_thickness_inverse, ax=axes[0], title='Predicted Thickness', label=plot_label, vmin=0, vmax=7)
    plot_array(target_thickness_inverse, ax=axes[1], title='Target Thickness', label=plot_label)
    plot_array(predicted_thickness_inverse - target_thickness_inverse, ax=axes[2], title='Thickness Difference', label=plot_label, cmap='coolwarm', vmin=-1, vmax=1)
    plt.tight_layout()
    plt.show() 

    # Plot the predicted velocity, target velocity, and their difference
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    plot_array(predicted_velocity_inverse, ax=axes[0], title='Predicted Velocity', label=plot_label)
    plot_array(target_velocity_inverse, ax=axes[1], title='Target Velocity', label=plot_label)
    plot_array(predicted_velocity_inverse - target_velocity_inverse, ax=axes[2], title='Velocity Difference', label=plot_label, cmap='coolwarm', vmin=-1, vmax=1)
    plt.tight_layout()
    plt.show()

    # # Inverse transform the predicted and target values using median and IQR
    # predicted_thickness_inverse = inverse_transform(predicted[..., 0], median_vals['thickness'], iqr_vals['thickness'])
    # target_thickness_inverse = inverse_transform(target[..., 0], median_vals['thickness'], iqr_vals['thickness'])
    # predicted_velocity_inverse = inverse_transform(predicted[..., 1], median_vals['velocity'], iqr_vals['velocity'])
    # target_velocity_inverse = inverse_transform(target[..., 1], median_vals['velocity'], iqr_vals['velocity'])


    # # Plot the predicted thickness, target thickness, and their difference
    # fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    # plot_array(predicted_thickness_inverse, ax=axes[0], title='Predicted Thickness', label=plot_label, vmin=0, vmax=7)
    # plot_array(target_thickness_inverse, ax=axes[1], title='Target Thickness', label=plot_label)
    # plot_array(predicted_thickness_inverse - target_thickness_inverse, ax=axes[2], title='Thickness Difference', label=plot_label, cmap='coolwarm', vmin=-1, vmax=1)
    # plt.tight_layout()
    # plt.show()


    # # Plot the predicted velocity, target velocity, and their difference
    # fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    # plot_array(predicted_velocity_inverse, ax=axes[0], title='Predicted Velocity', label=plot_label)
    # plot_array(target_velocity_inverse, ax=axes[1], title='Target Velocity', label=plot_label)
    # plot_array(predicted_velocity_inverse - target_velocity_inverse, ax=axes[2], title='Velocity Difference', label=plot_label, cmap='coolwarm', vmin=-1, vmax=1)
    # plt.tight_layout()
    # plt.show()



10


KeyboardInterrupt: 